In [2]:
pip install --upgrade boto3 botocore


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 2.1 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 45.1 MB/s eta 0:00:00:00:010:01
  Attempting uninstall: botocore
    Found existing installation: botocore 1.34.84
    Uninstalling botocore-1.34.84:
      Successfully uninstalled botocore-1.34.84
  Attempting uninstall: boto3
    Found existing installation: boto3 1.34.84
    Uninstalling boto3-1.34.84:
      Successfully uninstalled boto3-1.34.84
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
awscli 1.32.84 requires botocore==1.34.84, but you have botocore 1.35.63 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [3]:
import os
import boto3
import sagemaker
from datetime import date
from sagemaker.workflow.parallelism_config import ParallelismConfiguration
import utils
import pandas as pd
from production.pipelines_code.pipeline import get_pipeline as pipeline_definition
import datetime
from calendar import monthrange

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


In [4]:
# Specify the path you want to change to
new_directory = '/root/NPS forecasting model/nps_client_model/Client simulation/Simulation pipeline'

# Change the current working directory
os.chdir(new_directory)

# Verify the change
current_directory = os.getcwd()
print("Current Directory:", current_directory)
      
escenarios = ["all_changes"]


Current Directory: /root/NPS forecasting model/nps_client_model/Client simulation/Simulation pipeline


In [5]:
data_df={}
for escenario in escenarios:
    # data_df[escenario] = pd.read_csv(f'develop/FP 2025/{escenario}/Input_Operative_Iniciatives_20241107.csv', delimiter=';')
    data_df[escenario] = pd.read_csv(f'develop/Cabin haul level sims/yearly_NPS_2024-11-15.csv', delimiter=',')
    
    # Convert 'date_flight_local' to datetime if necessary
    data_df[escenario]['date_flight_local'] = pd.to_datetime(data_df[escenario]['date_flight_local'])

    #Filter year
    # data_df[escenario] = data_df[escenario][data_df[escenario]['company'] == 'IB']
    # data_df[escenario] = data_df[escenario][data_df[escenario]['year'] == '2025 Punctuality']
    
    data_df[escenario] = data_df[escenario][data_df[escenario]['operating_airline_code'] == 'IB']
    data_df[escenario] = data_df[escenario][data_df[escenario]['date_flight_local'].dt.year == 2024]

    # Round all numeric columns to 1 decimal place
    data_df[escenario] = data_df[escenario].round(1)


    variable_names = [
        "ticket_price",
        "load_factor",
        "pun_100_punctuality_satisfaction",
        "bkg_200_journey_preparation_satisfaction",
        "pfl_100_checkin_satisfaction",
        "pfl_200_security_satisfaction",
        "pfl_300_lounge_satisfaction",
        "pfl_500_boarding_satisfaction",
        "ifl_300_cabin_satisfaction",
        "ifl_200_flight_crew_annoucements_satisfaction",
        "ifl_600_wifi_satisfaction",
        "ifl_500_ife_satisfaction",
        "ifl_400_food_drink_satisfaction",
        "ifl_100_cabin_crew_satisfaction",
        "arr_100_arrivals_satisfaction",
        "con_100_connections_satisfaction",
        "loy_200_loyalty_programme_satisfaction",
        "img_310_ease_contact_phone_satisfaction"
    ]
    
    data_df[escenario] = data_df[escenario][['date_flight_local', 'cabin', 'haul'] + variable_names]
    
    # Extract 'YYYY-MM' from 'date_flight_local' and create a new 'month' column
    data_df[escenario]['month'] = data_df[escenario]['date_flight_local'].dt.month

    # Define months from January to December 2024
    months = data_df[escenario]['month'].unique()
    
    # data_df[escenario].to_csv(f'targets_puntuality_fp_25.csv')
    
    display(data_df[escenario])

,date_flight_local,cabin,haul,ticket_price,load_factor,pun_100_punctuality_satisfaction,bkg_200_journey_preparation_satisfaction,pfl_100_checkin_satisfaction,pfl_200_security_satisfaction,pfl_300_lounge_satisfaction,...,ifl_200_flight_crew_annoucements_satisfaction,ifl_600_wifi_satisfaction,ifl_500_ife_satisfaction,ifl_400_food_drink_satisfaction,ifl_100_cabin_crew_satisfaction,arr_100_arrivals_satisfaction,con_100_connections_satisfaction,loy_200_loyalty_programme_satisfaction,img_310_ease_contact_phone_satisfaction,month
23,2024-12-31,Business,LH,1336.2,209.7,80.7,73.3,80.9,83.6,76.8,...,82.3,45.3,71.8,70.3,82.8,79.3,68.4,72.6,56.8,12
26,2024-12-31,Business,SH,225.6,177.9,76.2,74.7,80.8,82.0,72.3,...,80.1,50.4,71.5,76.3,87.6,78.3,69.3,69.0,50.3,12
32,2024-12-31,Economy,LH,500.8,211.2,76.9,67.7,74.9,82.2,76.7,...,80.6,44.6,75.4,60.3,75.8,76.8,66.9,63.4,50.9,12
35,2024-12-31,Economy,SH,116.2,171.2,70.5,67.7,75.2,77.8,71.8,...,76.2,41.4,71.5,49.8,77.5,72.5,63.4,63.2,48.1,12
41,2024-12-31,Premium Economy,LH,748.9,209.5,80.1,69.7,77.4,83.1,80.9,...,80.0,45.6,74.4,57.1,74.1,78.0,67.0,70.2,49.4,12


In [6]:
# Define execution dates in August 2024 corresponding to the month numbers
execution_date = '2024-11-15'

# Extract unique cabins and hauls from your DataFrame
cabins = ['Economy']
hauls = ['SH']

# By default
region = boto3.Session().region_name
role = sagemaker.get_execution_role()

# Use own bucket and prefix
s3_bucket_nps = 'iberia-data-lake'  # In this case: iberia-data-lake
s3_bucket_nps_prefix = 'customer/nps_explainability_model'  # In this case: sagemaker/sagemaker-template

s3_bucket_lf = 'ibdata-prod-ew1-s3-customer'
s3_bucket_lf_prefix = 'customer/load_factor_to_s3_nps_model'

# Environment (sbx or prod pipeline)
environment = "develop"
environment_ppl = "prod" if environment == "production" else "sbx"

# Config file
configuration = utils.read_config_data()

# Parameters
is_last_date = "0"  # Optional
model_version = configuration.get("MODEL_VERSION")
vertical = configuration.get("VERTICAL")

s3_path_read_nps = 'customer/nps_surveys/export_historic'
s3_path_read_lf = "customer/load_factor_to_s3_nps_model"
use_type = 'Client'


In [7]:
# Import your pipeline definition
from production.pipelines_code.pipeline import get_pipeline as pipeline_definition

pipeline_name = f"ibdata-{vertical}-{model_version.lower()}-{environment_ppl}-ppl"
base_job_prefix = pipeline_name

# Initialize the pipeline
pipe_definition = pipeline_definition(
    region=region,
    role=role,
    default_bucket='iberia-data-lake',
    default_bucket_prefix='customer/simulations/sbx/pipeline',
    pipeline_name=pipeline_name,
    base_job_prefix=base_job_prefix
)
pipe_definition.upsert(role_arn=role)

INFO:sagemaker.processing:Uploaded None to s3://iberia-data-lake/customer/simulations/sbx/pipeline/ibdata-customer-simulations-sbx-ppl/fra-2024-11-18-11-52-59-886/source/sourcedir.tar.gz
INFO:sagemaker.processing:runproc.sh uploaded to s3://iberia-data-lake/customer/simulations/sbx/pipeline/ibdata-customer-simulations-sbx-ppl/fra-2024-11-18-11-52-59-886/source/runproc.sh
INFO:sagemaker.processing:Uploaded None to s3://iberia-data-lake/customer/simulations/sbx/pipeline/ibdata-customer-simulations-sbx-ppl/fra-2024-11-18-11-53-00-090/source/sourcedir.tar.gz
INFO:sagemaker.processing:runproc.sh uploaded to s3://iberia-data-lake/customer/simulations/sbx/pipeline/ibdata-customer-simulations-sbx-ppl/fra-2024-11-18-11-53-00-090/source/runproc.sh
INFO:sagemaker.processing:Uploaded None to s3://iberia-data-lake/customer/simulations/sbx/pipeline/ibdata-customer-simulations-sbx-ppl/fra-2024-11-18-11-53-00-175/source/sourcedir.tar.gz
INFO:sagemaker.processing:runproc.sh uploaded to s3://iberia-data

{'PipelineArn': 'arn:aws:sagemaker:eu-west-1:077156906314:pipeline/ibdata-customer-simulations-sbx-ppl',
 'ResponseMetadata': {'RequestId': 'b35cd985-e72a-43b0-8b75-55765a76dde7',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'b35cd985-e72a-43b0-8b75-55765a76dde7',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '103',
   'date': 'Mon, 18 Nov 2024 11:53:01 GMT'},
  'RetryAttempts': 0}}

In [8]:
import datetime
from calendar import monthrange

# Define the year range and cutoff date
year = 2023
cutoff_date = datetime.date(2023, 11, 15)
start_date = datetime.date(year, 1, 1)
end_date = cutoff_date

str_execution_date = execution_date
insert_date_ci = str_execution_date

# Format the start and end dates
str_start_date = start_date.strftime('%Y-%m-%d')
str_end_date = end_date.strftime('%Y-%m-%d')


# Initialize an empty list for the combined DataFrame
df_combined = []

for cabin in cabins:
    for haul in hauls:
        # Prepare the S3 path without the {month}
        s3_path_write = f"customer/simulations/sbx/FP/{escenario}/{use_type}/{cabin}_{haul}/2024"
        for escenario in escenarios:
            
            # Filter the data for the current combination over the entire period
            df_filtered = data_df[escenario][
                (data_df[escenario]['cabin'] == cabin) &
                (data_df[escenario]['haul'] == haul)
            ]

            if df_filtered.empty:
                print(f"No data for cabin {cabin}, haul {haul}")
                continue

            # Drop unnecessary columns
            df_targets = df_filtered.drop(columns=[
                'date_flight_local', 'month', 'cabin', 'haul', 'year'
            ], errors='ignore')
            
            df_targets_str = df_targets.to_csv(index=False, header=False).strip().replace('\n', '')

            # Start the pipeline execution
            execution = pipe_definition.start(
                parallelism_config=ParallelismConfiguration(
                    max_parallel_execution_steps=4
                ).to_request(),
                parameters=dict(
                    str_execution_date=str_execution_date,
                    is_last_date=is_last_date,
                    s3_path_write=s3_path_write,
                    s3_path_read_nps=s3_path_read_nps,
                    s3_path_read_lf=s3_path_read_lf,
                    str_start_date=str_start_date,
                    str_end_date=str_end_date,
                    str_cabin=cabin,
                    str_haul=haul,
                    df_targets=df_targets_str,
                    use_type=use_type
                )
            )
            # Optionally, describe the execution
            print(f"Started pipeline for cabin {cabin}, haul {haul}, from {start_date} to {end_date} execution date {str_execution_date}")
            execution.describe()


Started pipeline for cabin Economy, haul SH, from 2023-01-01 to 2023-11-15 execution date 2024-11-15


In [9]:
str_end_date

'2023-11-15'